# LangChain Experimentation

Experiment with LangChain and OpenAI API.

In [85]:
# Import Standard Libraries
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

from langchain.chains import LLMChain

# OpenAI API

In [4]:
# Set OpenAI API Key
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
def get_inference(prompt: str, 
                  model: str = 'gpt-3.5-turbo') -> str:
    
    # Compose the request body
    messages = [
        {'role': 'user',
         'content': prompt}]
    
    # Send the request and retrieve the response
    response = openai.ChatCompletion.create(model=model, 
                                            messages=messages, 
                                            temperature=0)
    
    return response.choices[0].message['content']

In [6]:
get_inference('What is 1+1?')

RateLimitError: You exceeded your current quota, please check your plan and billing details.

# LangChain

## Open AI Chat

In [47]:
# Instantiate a ChatOpenAI object
# NOTE: temperature at 0.0 reduces the noise, but also generalization capabilities
# Note2: it retrieves the API KEY from 'openai.api_key'
langchain_chat = ChatOpenAI(temperature=0.0)

## Prompt Template

In [39]:
# Define prompt template string
string_prompt_template = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [40]:
# Initialise a Prompt Template
langchain_prompt_template = ChatPromptTemplate.from_template(string_prompt_template)

In [41]:
langchain_prompt_template

ChatPromptTemplate(input_variables=['text', 'style'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [42]:
# Retrieve prompt input variables
langchain_prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [43]:
# Define a prompt style
prompt_style = """American english \
in a calm and respectful tone.
"""

In [44]:
# Define the prompt text
prompt_text = """Arr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need your help \
right now, mattey!
"""

In [45]:
# Generate the message from the prompt style and text
message = langchain_prompt_template.format_messages(style=prompt_style, text=prompt_text)

In [46]:
message[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American english in a calm and respectful tone.\n. text: ```Arr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need your help right now, mattey!\n```\n", additional_kwargs={}, example=False)

In [48]:
# Get the response
message_response = langchain_chat(message)

AuthenticationError: Incorrect API key provided: sk-fMYBA***************************************D4yg. You can find your API key at https://platform.openai.com/account/api-keys.

## Parser

It is used to extract information from LLM’s output in a machine-readable format (e.g., JSON from LLM’s output text).

In [49]:
# Expected JSON output
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [50]:
# Input customer review
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# Prompt tempalte
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [51]:
# Create prompt template
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [53]:
# Query the LLM
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
#response = chat(messages)
#print(response.content)

In [54]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
#response.content.get('gift')

In [56]:
# Define a set of schemas to extract the information from the customer review
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [57]:
# Instantiate the Output Parser from the above define schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [61]:
# Reitreve instructions that the prompt will pass to the LLM to have an output able to be parsed
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [62]:
# Define new prompt
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

# Define new prompt template
prompt = ChatPromptTemplate.from_template(template=review_template_2)

# Create a message with the format instruction
messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [64]:
# Query the LLM
#response = langchain_chat(messages)

In [63]:
# Parse the output
#output_dict = output_parser.parse(response.content)

## Conversation Buffer Memory

It helps to feed to the LLM the history of the current conversation.

In [67]:
# Define an Open AI Chat
llm = ChatOpenAI(temperature=0.0)

# Create a Conversation Memory
memory = ConversationBufferMemory()

# Create a Conversation that allows to interact with the LLM while keeping the memory of the conversation
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

In [68]:
# First prompt
#conversation.predict(input="Hi, my name is Simone")

In [69]:
# Intermediate prompt
#conversation.predict(input="What is 1+1?")

In [70]:
# Final prompt (it remembers my name)
#conversation.predict(input="What is my name?")

By setting `verbose=True` the ChatOpenAI object will feed the history of the conversation every time as part of the next prompt.

In [71]:
#print(memory.buffer)

The above object is keeping track of the conversation history.

In [72]:
# It is possible to restore the conversation into another ConversationBufferMemory
memory = ConversationBufferMemory()

# Restore
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

print(memory.buffer)

Human: Hi
AI: What's up


In [73]:
# Top up
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [74]:
print(memory.buffer)

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool


## Conversation Buffer Window Memory

Upon sending each time the whole conversation history to LLM, the prompt starts to begin quite long in terms of tokens (and thus expensive). The Conversation Buffer Window Memory allows to keep just a subset of the whole conversation. The last `k` interactions.

In [76]:
# Instantiate a Conversation Window of one history
memory = ConversationBufferWindowMemory(k=1)               

In [77]:
# Set history
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [78]:
# Only last interaction is kept
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

## Conversation Token Buffer Memory

It remembers only the last k tokens of the conversation.

In [81]:
# Setup the conversation token buffer
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)

# Set history
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [82]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## Conversation Summary Memory

It uses the LLM to write a summary of the conversation so far!

In [84]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# Define the Conversation Summary
memory = ConversationSummaryBufferMemory(llm=llm, 
                                         max_token_limit=100)

# Set history
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

AuthenticationError: Incorrect API key provided: sk-fMYBA***************************************D4yg. You can find your API key at https://platform.openai.com/account/api-keys.

## Chains

A chain is a combination of a LLM with a generic input prompt. It is possible to combine input/output sequence of LLMs prompts and related outputs.

### Sequential Chains

The idea is to combine multiple chgains where the output of one is the input of the next one.


**Types:**
- Simple Sequential Chain: Single input/output
- Sequential Chain: Multiple inputs/outputs

In [86]:
# Define LLM
llm = ChatOpenAI(temperature=0.9)

In [87]:
# Define prompt
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [89]:
prompt.input_variables

['product']

In [91]:
# Chain LLM and Prompt
chain = LLMChain(llm=llm, prompt=prompt)

In [93]:
# Execute the chain: pass the prompt to the LLM
product = "Queen Size Sheet Set"
#chain.run(product) # Output: "Royal Comfort Linens"